In [ ]:
pip install boto3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import boto3
from packs.env import NCP_ACCESS_KEY, NCP_SECRET_KEY

service_name = 's3'
endpoint_url = 'https://kr.object.ncloudstorage.com'
region_name = 'kr-standard'
access_key = NCP_ACCESS_KEY
secret_key = NCP_SECRET_KEY
file_list = []

if __name__ == "__main__":
    s3 = boto3.client(service_name, endpoint_url=endpoint_url, aws_access_key_id=access_key,
                      aws_secret_access_key=secret_key)

    bucket_name = 'loldata'

    # list all in the bucket
    max_keys = 300
    folder_prefix ='matches/'

    # top level folders and files in the bucket
    delimiter = '/'
    max_keys = 300

    response = s3.list_objects(Bucket=bucket_name,Prefix=folder_prefix, Delimiter=delimiter, MaxKeys=max_keys)

    print('top level folders and files in the bucket')

    while True:

        for content in response.get('Contents'):
            # print(' Name=%s, Size=%d, Owner=%s' % \
            #       (, content.get('Size'), content.get('Owner').get('ID')))
            file_list.append(content.get('Key'))

        if response.get('IsTruncated'):
            response = s3.list_objects(Bucket=bucket_name, Delimiter=delimiter, MaxKeys=max_keys,
                                       Marker=response.get('NextMarker'))
        else:
            break

top level folders and files in the bucket


In [ ]:
len(file_list[1:])

22

In [ ]:
import pandas as pd
from io import StringIO

for index, object_key in enumerate(file_list[1:]):
  response = s3.get_object(Bucket=bucket_name, Key=object_key)
  csv_data = response['Body'].read().decode('utf-8')
  df = pd.read_csv(StringIO(csv_data))
  if index== 0:
    merged_df = df
  elif index == len(file_list[1:])-1:
    eval_df = df
  else:
    merged_df = pd.concat([merged_df, df], ignore_index=True)

In [ ]:
### id, champname, dim map
import ast

raw_dict = {}

for _, row in merged_df.iterrows():
    id_list = ast.literal_eval (row['blueChampIds'])
    name_list = ast.literal_eval(row['blueChampNames'])

    # 각 id와 name을 매핑
    for id_value, name in zip(id_list, name_list):
        raw_dict[name] = id_value

for i in range(200 - len(raw_dict)):
    raw_dict[f'unnamed{i}'] = -i - 1

name_id_dict = {k: raw_dict[k] for k in sorted(raw_dict)}
id_name_dict = {value: key for key, value in name_id_dict.items()}
sorted_keys = sorted(name_id_dict.keys())
name_dim_dict = {sorted_keys[i]: i for i in range(len(sorted_keys))}
dim_name_dict = {value: key for key, value in name_dim_dict.items()}

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import math
from torch.optim.lr_scheduler import CosineAnnealingLR, LambdaLR

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class ValueNetwork(nn.Module):
    def __init__(self, num_players, embedding_dim, hidden_size, output_size):
        super(ValueNetwork, self).__init__()
        self.embedding = nn.Embedding(num_players, embedding_dim)  # num_players: 고유 선수 이름 수
        init.kaiming_uniform_(self.embedding.weight, a=math.sqrt(5))

        self.fc1 = nn.Linear(embedding_dim * 10, hidden_size)  # 각 팀에 5명의 선수
        self.fc2 = nn.Linear(hidden_size, hidden_size)  # 출력: 승패 예측
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, hidden_size)
        self.fc5 = nn.Linear(hidden_size, hidden_size)
        self.fc6 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, team1, team2):
        team1_embedding = self.embedding(team1)  # 5명의 선수 상태
        team2_embedding = self.embedding(team2)

        # 팀의 상태 벡터를 결합
        combined = torch.cat((team1_embedding.view(-1), team2_embedding.view(-1)))  # 10차원 벡터

        x = self.fc1(combined)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        x = self.relu(x)
        x = self.fc5(x)
        x = self.relu(x)
        x = self.fc6(x)
        return x

def cosine_with_warmup_scheduler(optimizer, num_warmup_steps, num_training_steps):
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))  # Linear warmup
        else:
            cosine_decay = 0.5 * (1 + torch.cos(
                torch.pi * torch.tensor((current_step - num_warmup_steps) / (num_training_steps - num_warmup_steps))
            ))
            return float(cosine_decay)

    return LambdaLR(optimizer, lr_lambda)

In [ ]:
num_players = len(name_dim_dict) # 챔프의 수
embedding_dim = 20  # 임베딩 차원 (선택 가능)
hidden_size = 64  # 숨겨진 층 크기
output_size = 1  # 출력 크기 (승패 예측)

model = ValueNetwork(num_players, embedding_dim, hidden_size, output_size).to(device)
criterion = nn.MSELoss()  # 평균 제곱 오차
optimizer = optim.Adam(model.parameters(), lr=0.0001)
epochs = 10

scheduler = cosine_with_warmup_scheduler(optimizer,int(len(merged_df) * epochs/5) ,len(merged_df) * epochs)

In [ ]:
def trash_games(lane):
  order = [0,1,2,3,4]
  response = True
  # check all lanes are there
  if set(lane) !=  set(['TOP', 'JUNGLE', 'MIDDLE', 'BOTTOM', 'UTILITY']):
    response = False
  else:
  #check right order
    target_order = ['TOP', 'JUNGLE', 'MIDDLE', 'BOTTOM', 'UTILITY']
    order = [lane.index(value) for value in target_order]

  return order, response

In [ ]:
model

ValueNetwork(
  (embedding): Embedding(200, 20)
  (fc1): Linear(in_features=200, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=64, bias=True)
  (fc6): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
)

In [ ]:
# 학습 후 예측

def eval_models(model):
  model.eval()
  correct,false, normal_correct, normal_false, strong_correct, strong_false = [], [], [], [], [], []
  with torch.no_grad():
      for _, row in eval_df.iterrows():
        blue_champs = ast.literal_eval(row['blueChampNames'])
        red_champs = ast.literal_eval(row['redChampNames'])
        blue_lanes = ast.literal_eval(row['blueLanes'])
        red_lanes = ast.literal_eval(row['redLanes'])
        blue_wins = row['blueWin']
        #불량 경기 체크
        order, response = trash_games(blue_lanes)
        if response == False:
          continue
        if order != [0,1,2,3,4]:
          blue_champs = [blue_champs[i] for i in order]

        order, response = trash_games(red_lanes)
        if response == False:
          continue
        if order != [0,1,2,3,4]:
          red_champs = [red_champs[i] for i in order]

        test_team1_indices = torch.tensor([name_dim_dict[name] for name in blue_champs], dtype=torch.long).to(device)
        test_team2_indices = torch.tensor([name_dim_dict[name] for name in red_champs], dtype=torch.long).to(device)

        predicted_value = model(test_team1_indices, test_team2_indices)
        if predicted_value > merged_df['blueWin'].mean(): # bluewin 판정
          check = True
        else: #redwin 판정
          check = False
        if predicted_value > 0.5: # bluewin 판정
          normal_check = True
          if blue_wins and predicted_value > 0.67:
            strong_correct.append(predicted_value)
        else: #redwin 판정
          normal_check = False
          if (not blue_wins) and predicted_value<0.23:
            strong_false.append(predicted_value)

        if check == blue_wins:
          correct.append(predicted_value)
        else:
          false.append(predicted_value)

        if normal_check == blue_wins:
          normal_correct.append(predicted_value)
        else:
          normal_false.append(predicted_value)


  results = correct + false
  normal_results = normal_correct + normal_false
  strong_results = strong_correct + strong_false
  accuracy = len(correct) / len(results) * 100  # 정답 개수 / 전체 개수 * 100
  orig_accuracy = len(normal_correct) / len(normal_results) * 100
  if len(strong_results)== 0:
    strong_results.append('nothing')
  strong_accuracy = len(strong_correct) / len(strong_results) * 100
  print(f"정답률: {accuracy:.2f}%")
  print(f"보정없는정답률: {orig_accuracy:.2f}%")
  print(f"확실한경기만 정답률: {strong_accuracy:.2f}%")
  print(f"확실한경기 비율: {len(strong_results)/len(results):.2f}%")

In [ ]:
from tqdm import tqdm
from collections import deque

recent_losses = deque(maxlen=1000)
# 학습 루프

for epoch in range(epochs):
    model.train()
    with tqdm(merged_df.iterrows(), total=merged_df.shape[0], desc="Processing rows") as pbar:
      for step, row in pbar:
        blue_champs = ast.literal_eval(row['blueChampNames'])
        red_champs = ast.literal_eval(row['redChampNames'])
        blue_lanes = ast.literal_eval(row['blueLanes'])
        red_lanes = ast.literal_eval(row['redLanes'])
        blue_wins = row['blueWin']

        #불량 경기 체크
        order, response = trash_games(blue_lanes)
        if response == False:
          continue
        if order != [0,1,2,3,4]:
          blue_champs = [blue_champs[i] for i in order]

        order, response = trash_games(red_lanes)
        if response == False:
          continue
        if order != [0,1,2,3,4]:
          red_champs = [red_champs[i] for i in order]



        team1_indices = torch.tensor([name_dim_dict[name] for name in blue_champs], dtype=torch.long).to(device)
        team2_indices = torch.tensor([name_dim_dict[name] for name in red_champs], dtype=torch.long).to(device)

        if blue_wins:
          target_value = torch.tensor([1.0], device= device)
        else:
          target_value = torch.tensor([0.0], device = device)
        # 순전파
        predicted_value = model(team1_indices, team2_indices)  # 예측된 승패

        # 손실 계산
        loss = criterion(predicted_value, target_value)

        # 기울기 초기화, 역전파, 최적화
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        recent_losses.append(loss.item())
        if step %1000 == 0:
          avg_loss = sum(recent_losses) / len(recent_losses)

          pbar.set_postfix({"epochs": epoch, "Loss":avg_loss, "LR": scheduler.get_last_lr()[0]})
      # 학습 진행 상황 출력
    eval_models(model)



Processing rows: 100%|██████████| 32499/32499 [01:28<00:00, 367.09it/s, epochs=0, Loss=0.249, LR=4.76e-5]


정답률: 52.22%
보정없는정답률: 54.05%
확실한경기만 정답률: 0.00%
확실한경기 비율: 0.00%


Processing rows: 100%|██████████| 32499/32499 [01:26<00:00, 374.92it/s, epochs=1, Loss=0.247, LR=9.6e-5]


정답률: 52.92%
보정없는정답률: 54.05%
확실한경기만 정답률: 100.00%
확실한경기 비율: 0.00%


Processing rows: 100%|██████████| 32499/32499 [01:30<00:00, 359.79it/s, epochs=2, Loss=0.238, LR=9.7e-5]


...

정답률: 53.70%
보정없는정답률: 53.07%
확실한경기만 정답률: 71.70%
확실한경기 비율: 0.34%


Processing rows: 100%|██████████| 32499/32499 [01:31<00:00, 356.52it/s, epochs=8, Loss=0.144, LR=6.43e-6]


정답률: 52.92%
보정없는정답률: 52.85%
확실한경기만 정답률: 70.88%
확실한경기 비율: 0.33%


Processing rows: 100%|██████████| 32499/32499 [01:31<00:00, 355.73it/s, epochs=9, Loss=0.139, LR=4.39e-7]


정답률: 52.50%
보정없는정답률: 52.57%
확실한경기만 정답률: 70.60%
확실한경기 비율: 0.30%


In [ ]:
model.eval()
correct,false, normal_correct, normal_false, strong_correct, strong_false = [], [], [], [], [], []
with torch.no_grad():
    for _, row in eval_df.iterrows():
      blue_champs = ast.literal_eval(row['blueChampNames'])
      red_champs = ast.literal_eval(row['redChampNames'])
      blue_lanes = ast.literal_eval(row['blueLanes'])
      red_lanes = ast.literal_eval(row['redLanes'])
      blue_wins = row['blueWin']
      #불량 경기 체크
      order, response = trash_games(blue_lanes)
      if response == False:
        continue
      if order != [0,1,2,3,4]:
        blue_champs = [blue_champs[i] for i in order]

      order, response = trash_games(red_lanes)
      if response == False:
        continue
      if order != [0,1,2,3,4]:
        red_champs = [red_champs[i] for i in order]

      test_team1_indices = torch.tensor([name_dim_dict[name] for name in blue_champs], dtype=torch.long).to(device)
      test_team2_indices = torch.tensor([name_dim_dict[name] for name in red_champs], dtype=torch.long).to(device)

      predicted_value = model(test_team1_indices, test_team2_indices)
      if predicted_value > merged_df['blueWin'].mean(): # bluewin 판정
        check = True
      else: #redwin 판정
        check = False
      if predicted_value > 0.5: # bluewin 판정
        normal_check = True
        if blue_wins and predicted_value > 0.7:
          strong_correct.append(predicted_value)
      else: #redwin 판정
        normal_check = False
        if (not blue_wins) and predicted_value<0.2:
          strong_false.append(predicted_value)

      if check == blue_wins:
        correct.append(predicted_value)
      else:
        false.append(predicted_value)

      if normal_check == blue_wins:
        normal_correct.append(predicted_value)
      else:
        normal_false.append(predicted_value)


results = correct + false
normal_results = normal_correct + normal_false
strong_results = strong_correct + strong_false
accuracy = len(correct) / len(results) * 100  # 정답 개수 / 전체 개수 * 100
orig_accuracy = len(normal_correct) / len(normal_results) * 100
if len(strong_results)== 0:
  strong_results.append('nothing')
strong_accuracy = len(strong_correct) / len(strong_results) * 100
print(f"정답률: {accuracy:.2f}%")
print(f"보정없는정답률: {orig_accuracy:.2f}%")
print(f"확실한경기만 정답률: {strong_accuracy:.2f}%")
print(f"확실한경기 비율: {len(strong_results)/len(results):.2f}%")

정답률: 52.50%
보정없는정답률: 52.57%
확실한경기만 정답률: 70.83%
확실한경기 비율: 0.25%


In [ ]:
len(strong_results)

360

In [ ]:
save_path = '/content/drive/MyDrive/lol_model.pth'

# 모델 저장
torch.save(model.state_dict(), save_path)

In [ ]:
eval_models(model)

정답률: 52.50%
보정없는정답률: 52.57%
확실한경기만 정답률: 70.60%
확실한경기 비율: 0.30%


In [ ]:
correct

[tensor([0.5854], device='cuda:0'),
 tensor([0.2170], device='cuda:0'),
 tensor([-0.0033], device='cuda:0'),
 tensor([0.2922], device='cuda:0'),
 tensor([0.6800], device='cuda:0'),
 tensor([0.8727], device='cuda:0'),
 tensor([0.4519], device='cuda:0'),
 tensor([0.5853], device='cuda:0'),
 tensor([0.4147], device='cuda:0'),
 tensor([0.1627], device='cuda:0'),
 tensor([0.3931], device='cuda:0'),
 tensor([0.4077], device='cuda:0'),
 tensor([0.5165], device='cuda:0'),
 tensor([0.8085], device='cuda:0'),
 tensor([0.4228], device='cuda:0'),
 tensor([0.7564], device='cuda:0'),
 tensor([0.1325], device='cuda:0'),
 tensor([0.6162], device='cuda:0'),
 tensor([0.7052], device='cuda:0'),
 tensor([-0.0010], device='cuda:0'),
 tensor([0.7402], device='cuda:0'),
 tensor([0.6728], device='cuda:0'),
 tensor([0.4181], device='cuda:0'),
 tensor([0.4085], device='cuda:0'),
 tensor([0.6666], device='cuda:0'),
 tensor([0.8216], device='cuda:0'),
 tensor([0.8011], device='cuda:0'),
 tensor([0.6448], device='

In [ ]:
# model.eval()
# with torch.no_grad():
#   blue_champs= ['Galio','Galio','Galio','Galio','Galio']
#   red_champs= ['Galio','Galio','Galio','Galio','Galio']
#   test_team1_indices = torch.tensor([name_dim_dict[name] for name in blue_champs], dtype=torch.long).to(device)
#   test_team2_indices = torch.tensor([name_dim_dict[name] for name in red_champs], dtype=torch.long).to(device)

#   predicted_value = model(test_team1_indices, test_team2_indices)